In [1]:
import mlflow

mlflow.__version__

'2.21.3'

In [2]:
from src.utils.folder_operations import get_project_root

# set mlflow tracking uri
mlflow.set_tracking_uri(
    (get_project_root() / 'mlflow_new/mlruns').as_uri()
)

# Search runs

In [4]:
import random
from typing import Dict 
from src.utils.run_batch import create_runs_batch

## Generating Demo Runs

In [5]:
experiments = mlflow.search_experiments(view_type=mlflow.tracking.client.ViewType.ACTIVE_ONLY,
                                        max_results=3)

for experiment in experiments:
    create_runs_batch(experiment_id=experiment.experiment_id,n_runs=random.randint(1,5))

In [6]:
experiment_name = experiments[0].name
print(f"Experiment name: {experiment_name}")
runs = mlflow.search_runs(experiment_names=[experiment_name])

Experiment name: experiment_19


In [7]:
runs.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.metric_2,metrics.metric_1,params.param_2,params.param_1,tags.mlflow.note.content,tags.algorithm_type,tags.project_type,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.runName,tags.random_run,tags.mlflow.source.name
0,15ef683477e643a287c5cf70b1274593,111200603760646954,FINISHED,file:///Users/caiosainvallio/projects/mlflow_n...,2025-04-22 01:38:17.897000+00:00,2025-04-22 01:38:17.904000+00:00,0.894439,0.651697,z,c,This is a note,tree,experiment,caiosainvallio,LOCAL,bright-robin-599,yes,/Users/caiosainvallio/projects/mlflow_new/.ven...
1,fc0e85d05abb44909fb0136e3fba152e,111200603760646954,FINISHED,file:///Users/caiosainvallio/projects/mlflow_n...,2025-04-22 01:38:17.889000+00:00,2025-04-22 01:38:17.896000+00:00,0.780603,0.340763,z,b,This is a note,tree,experiment,caiosainvallio,LOCAL,adorable-tern-237,yes,/Users/caiosainvallio/projects/mlflow_new/.ven...
2,1ba980f60a574b1b9862358000e2051f,111200603760646954,FINISHED,file:///Users/caiosainvallio/projects/mlflow_n...,2025-04-22 01:38:17.880000+00:00,2025-04-22 01:38:17.888000+00:00,0.436237,0.742132,z,a,This is a note,tree,experiment,caiosainvallio,LOCAL,sedate-vole-857,yes,/Users/caiosainvallio/projects/mlflow_new/.ven...


# Using filter strings
In order to filter your MLflow runs, you will need to write search queries, which are pseudo-SQL conditions expressed in a distinct syntax.

In [8]:
runs = mlflow.search_runs(experiment_names=[experiment_name], filter_string="tags.project_type = 'development'")

In [9]:
runs.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time


In [10]:
runs[["metrics.metric_1","metrics.metric_2","params.param_1","params.param_2","tags.algorithm_type"]]

KeyError: "None of [Index(['metrics.metric_1', 'metrics.metric_2', 'params.param_1',\n       'params.param_2', 'tags.algorithm_type'],\n      dtype='object')] are in the [columns]"

In [11]:
runs = mlflow.search_runs(experiment_names=[experiment_name], filter_string="tags.project_type = 'development' AND metrics.metric_1 > 0.8")


In [12]:
runs[["metrics.metric_1","metrics.metric_2","params.param_1","params.param_2","tags.algorithm_type"]]


KeyError: "None of [Index(['metrics.metric_1', 'metrics.metric_2', 'params.param_1',\n       'params.param_2', 'tags.algorithm_type'],\n      dtype='object')] are in the [columns]"

# Returning Run Objects
To return Run objects we have to specify this using the parameter output_format

output_format – The output format to be returned. If pandas, a pandas.DataFrame is returned and, if list, a list of mlflow.entities.Run is returned.

In [13]:
runs = mlflow.search_runs(experiment_names=[experiment_name], filter_string="tags.project_type = 'development'", output_format="list")

In [14]:
for run in runs:
    print(type(run))
    print(f"Name: {run.info.run_name}, ID: {run.info.run_id}, Metrics: {run.data.metrics}")
    print("\n")